In [ ]:
##for a pair of odds where x and y are complements of each other i.e. duke winning vs UNC or UNC winning vs Duke
def odds_to_prob2(x, y):
    if (x < 0):
        prob_x = (-1 * x) / (100 - x)
    else:
        prob_x = 100 / (x + 100)
    if (y < 0):
        prob_y = (-1 * y) / (100 - y)
    else:
        prob_y = 100 / (x + 100)
    vig = prob_x + prob_y - 1.0
    prob_x -= vig/2
    prob_y -= vig/2
    return (prob_x, prob_y)

##Converting 
def odds_to_prob1(odds):
    if (odds < 0):
        return (-1 * odds) / (100 - odds)
    else:
        return 100 / (odds + 100)

##exp value payout for a bet size of $1
def calc_statistical_edge(odds, consensus_prob):
    if (odds < 0):
        payout = ((-100 / odds) + 1)
    else:
        payout = ((odds / 100) + 1)
    return payout * consensus_prob


def exp_value(odds, consensus_prob, principle):
    if (odds < 0):
        payout = ((-100 / odds) + 1) * principle
    else:
        payout = ((odds / 100) + 1) * principle
    return payout * consensus_prob

##If you want the statistical edge, just do the expected value for a principle of $1

def consensus_prob_calc(probs, weights=None):
    if not weights: # fallback value
        weights = [1/len(probs)] * len(probs)
    consensus_prob = 0
    for index in range(len(weights)):
        consensus_prob += weights[index] * probs[index]
    return consensus_prob

def odds_to_decimal_odds(odds):
    if (odds < 0):
        return ((-100 / odds) + 1)
    else:
        return ((odds / 100) + 1)

##Uses Kelly Criterion to calculate optimal bet size
def calc_bet_size(odds, consensus_prob, bankroll):
    decimal_odds = odds_to_decimal_odds(odds)
    frac = (decimal_odds - 1) * consensus_prob - ((1 - consensus_prob) / decimal_odds)
    if (frac < 0):
        return 0
    else:
        return frac * bankroll

: 

In [ ]:
import pandas as pd
import requests
import urllib
from datetime import date

# define API constants
BASE_URL = 'https://api.prop-odds.com'
API_KEY = 'GztX2ykLea7yEmXtJa1183dcc2JQ5AI9xXc5iHTKo' # enter your API key here (from https://prop-odds.com/profile)

def get_request(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()

    print('Request failed with status:', response.status_code)
    print(response.reason)
    return {}

: 

In [ ]:

historical_games = pd.read_csv('games_full_data.csv').dropna()
weights = {} # map sportsbook to weighting

: 

In [ ]:
def loss(game):
    """Penalizes the sportsbook based on how wrong their odds were
        Parameters:
            game (series): game data
        Returns:
            float: loss
    """
    if (game['home_team_odds'] < 0 and game['winner'] == game['home_team']) or (game['away_team_odds'] < 0 and game['winner'] == game['away_team']):
        return 0
    
    home_prob, away_prob = odds_to_prob2(game['home_team_odds'], game['away_team_odds'])
    return max(0, 1 - home_prob if (home_prob > away_prob) else 1 - away_prob)


: 

In [ ]:
def assign_weights():
    """Assigns weights to each sportsbook based on historical performance"""
    # assign loss to each game
    historical_games['loss'] = historical_games.apply(loss, axis=1)

    # calculate weights based on total loss for each sportsbook
    total_loss = historical_games.groupby('sportsbook')['loss'].sum()
    total_loss = total_loss / total_loss.sum()
    for sportsbook in total_loss.index:
        weights[sportsbook] = total_loss[sportsbook]

: 

In [ ]:
assign_weights()
weights

: 

In [ ]:
todays_games = pd.DataFrame(columns=['home_team', 'away_team', 'prop_odds_game_id'])
todays_odds = pd.DataFrame(columns=['home_team', 'away_team', 'prop_odds_game_id', 'sportsbook', 'home_team_odds', 'away_team_odds', 'probability', 'consensus_prob', 'exp_value'])

: 

In [ ]:
def get_todays_games():
    """Get today's games from the Prop Odds API"""
    query_params = {
        'date': date.today().strftime('%Y-%m-%d'),
        'tz': 'America/New_York',
        'api_key': API_KEY,
    }
    params = urllib.parse.urlencode(query_params)
    url = BASE_URL + '/beta/games/nba?' + params
    return get_request(url)

: 

In [ ]:
def fill_todays_games():
    """Fill the todays_games dataframe with today's games"""
    todays_games_json = get_todays_games()
    for game in todays_games_json['games']:
        todays_games.loc[len(todays_games)] = [game['home_team'], game['away_team'], game['game_id']]

: 

In [ ]:
def get_game_odds(game_id, market='moneyline'):
    """Returns an object containing the odds provided by the sportsbooks for the given game and market
        Parameters: 
            game_id (str): prop odds game id
            market (str): market to get odds for (moneyline, spread, total)
        Returns:
            odds: dict object containing the odds for the given game and market
    """
    query_params = {
        'api_key': API_KEY,
    }
    params = urllib.parse.urlencode(query_params)
    url = BASE_URL + '/beta/odds/' + game_id + '/' + market + '?' + params
    return get_request(url)

: 

In [ ]:
def identify_home_away(outcomes, home_team, away_team):
    """Given a list of outcome objects, identifies the home and away outcome objects
        Parameters:
            outcomes (list): list of outcome objects
            home_team (str): home team name
            away_team (str): away team name
        Returns:
            home_outcome: outcome object for the home team
            away_outcome: outcome object for the away team
    """
    home_outcome = None
    away_outcome = None
    home_team_name = home_team.split(' ')[-1]
    away_team_name = away_team.split(' ')[-1]
    for outcome in outcomes:
        if home_team_name in outcome['name']:
            home_outcome = outcome
        elif away_team_name in outcome['name']:
            away_outcome = outcome
    return home_outcome, away_outcome

: 

In [ ]:
def add_odds_for_single_game(game_id):
    """Adds odds for a single game to games_full_data for all available sportsbooks
        Parameters:
            game_id (str): prop odds game id
        Returns:
            None
    """
    odds = get_game_odds(game_id)
    if odds:
        for sportsbook in odds['sportsbooks']:
            sportsbook_name = sportsbook['bookie_key']
            game = todays_games[todays_games['prop_odds_game_id'] == game_id].copy()
            game['sportsbook'] = sportsbook_name
            home_outcome, away_outcome = identify_home_away(sportsbook['market']['outcomes'], game['home_team'].values[0], game['away_team'].values[0])
            game['home_team_odds'] = home_outcome['odds']
            game['away_team_odds'] = away_outcome['odds']
            game['probability'] = odds_to_prob2(home_outcome['odds'], away_outcome['odds'])[0]
            game['consensus_prob'], game['exp_value'] = None, None
            todays_odds.loc[len(todays_odds)] = game.values[0]
        

: 

In [ ]:
def fill_odds():
    """Calls add_odds_for_single_game for each game in games_df"""
    for game_id in todays_games['prop_odds_game_id']:
        add_odds_for_single_game(game_id)

: 

In [ ]:
def fill_consensus_probs():
    """Fills the consensus_prob column in todays_odds"""
    for game_id in todays_odds['prop_odds_game_id'].unique():
        game = todays_odds[todays_odds['prop_odds_game_id'] == game_id]
        consensus_prob = consensus_prob_calc(game['probability'].values)
        todays_odds.loc[todays_odds['prop_odds_game_id'] == game_id, 'consensus_prob'] = consensus_prob

: 

In [ ]:
def fill_exp_values():
    """Fills the exp_value column in todays_odds"""
    for index, row in todays_odds.iterrows():
        ev = exp_value(row['home_team_odds'], row['consensus_prob'], 1)
        todays_odds.loc[index, 'exp_value'] = ev

: 

In [ ]:
get_todays_games()
fill_todays_games()
fill_odds()
fill_consensus_probs()
fill_exp_values()
todays_odds.to_csv('todays_odds.csv', index=False)

: 